<a href="https://colab.research.google.com/github/samworkzz/sentiment_analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SENTIMENT ANALYSIS USING NLP**

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
#API to fetch data from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 162MB/s]
100% 80.9M/80.9M [00:00<00:00, 144MB/s]


In [4]:
#extracting zip file

from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("the dataset has been extracted")

the dataset has been extracted


EDA

In [5]:
#importing the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [9]:
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#naming the columns and reading the csv again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')


In [12]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [14]:
#checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [15]:
#converting the target '4' to '1'
twitter_data.replace({'target':{4:1}},inplace=True)

0->negative tweet
1->positive tweet


In [16]:
#stemming
port_stem=PorterStemmer()

In [17]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [18]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [19]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [20]:
#seperating the data and the label
X=twitter_data['stemmed_content'].values
y=twitter_data['target'].values

In [21]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [22]:
print(y)

[0 0 0 ... 1 1 1]


In [25]:
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [26]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [27]:
print(y.shape, y_train.shape, y_test.shape)

(1600000,) (1280000,) (320000,)


In [29]:
#converting the textual data to numerical
vectorizer= TfidfVectorizer()

X_train= vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [30]:
print(X_train)

  (0, 129956)	0.6592576617698773
  (0, 31233)	0.7519171067329923
  (1, 437910)	0.8042773910733999
  (1, 241525)	0.5942540519072339
  (2, 411086)	0.18464287904169105
  (2, 278606)	0.4853139770765464
  (2, 145727)	0.16444046079173225
  (2, 140234)	0.22372378686138533
  (2, 416219)	0.2970321685021026
  (2, 93639)	0.16612263205577082
  (2, 128288)	0.28170394757881806
  (2, 42838)	0.22940430929074537
  (2, 237072)	0.6367700241935804
  (3, 286683)	0.3682809811482607
  (3, 8560)	0.37668420201866026
  (3, 445747)	0.2188627839232472
  (3, 179097)	0.4788299006944713
  (3, 308910)	0.41292329558269175
  (3, 201928)	0.524207488360723
  (4, 227660)	0.3291530007938817
  (4, 415818)	0.27525076285252903
  (4, 315347)	0.3291134109458034
  (4, 302738)	0.35966531233864635
  (4, 457632)	0.7604081439946883
  (5, 275098)	0.27168172484322706
  :	:
  (1279996, 150554)	0.1621318868138012
  (1279996, 286184)	0.19295841733643207
  (1279996, 445242)	0.1645728738109807
  (1279997, 336356)	0.6816932944012981
  (1279

Training machine learning model

In [31]:
model= LogisticRegression(max_iter=10000)

In [32]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=10000)

Model Evaluation

In [33]:
X_train_prediction= model.predict(X_train)
training_data_accuracy=accuracy_score(y_train, X_train_prediction)

In [34]:
print("Accuracy score on the training data: ", training_data_accuracy)

Accuracy score on the training data:  0.81003359375


In [36]:
X_test_prediction= model.predict(X_test)
testing_data_accuracy=accuracy_score(y_test, X_test_prediction)

In [37]:
print("Accuracy score on the training data: ", testing_data_accuracy)


Accuracy score on the training data:  0.7783125


Model_accuracy=77.8%

**saving the trained model**

In [38]:
import pickle

In [39]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [40]:
#loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [44]:
X_new = X_test[3]
print(y_test[3])

prediction=loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("negative tweet")
else:
  print("positive tweet")

0
[1]
positive tweet
